In [1]:
from bs4 import BeautifulSoup as bs
import time
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
import matplotlib.pyplot as plt
import pandas as pd

In [5]:
# 크롤링할 웹사이트 주소
url = 'https://www.facebook.com/GyeonggiKorea.en'

# webdriver 로드
driver = webdriver.Chrome('chromedriver')
driver.get(url)

# 창 최대화
driver.maximize_window()
time.sleep(1)

In [6]:
# 페이지 스크롤 다운
num_pagedown = 3
for i in range(num_pagedown):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)

In [7]:
# 페이지 크롤링
soup = bs(driver.page_source, 'lxml')
soup.title.text

'Gyeonggi Korea | Facebook'

In [67]:
# 팔로워 수 수집
num_follower = soup.find('span', 'd2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em iv3no6db jq4qci2q a3bd9o3v b1v8xokw oo9gr5id hzawbc8m')
num_follower.text

'팔로워 9.9만명'

In [70]:
temp = num_follower.text[4:-2]
temp

'9.9'

In [71]:
temp2 = float(temp)
temp2 * 10000

99000.0

In [51]:
# 글 날짜 수집
info_dates_span = soup.find_all('a', 'oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl gmql0nx0 gpro0wi8 b1v8xokw')
info_date = info_dates_span[0].find('span')
info_date.text

'3월 4일 오후 6:21'

In [55]:
len(info_dates_span)

13

In [52]:
info_date = info_dates_span[1].find('span')
info_date.text

'3월 4일 오전 10:58'

In [54]:
info_date = info_dates_span[2].find('span')
info_date.text

'3월 2일 오후 5:04'

In [8]:
# 글 내용 크롤링
content_pages = soup.find_all('div', class_='kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x c1et5uql ii04i59q')

len(content_pages)

13

In [14]:
# 좋아요, 댓글, 공유 수 정보 나온 페이지 수집
info_pages = soup.find_all('div', 'bp9cbjyn m9osqain j83agx80 jq4qci2q bkfpd7mw a3bd9o3v kvgmc6g5 wkznzc2l oygrvhab dhix69tm jktsbyx5 rz4wbd8a osnr6wyh a8nywdso s1tcr66n')
len(info_pages)

13

In [56]:
# 좋아요 수 수집
num_like = info_pages[0].find('span', 'pcp91wgn')
int(num_like.text)

17

In [93]:
# 댓글 수 크롤링
num_comment = info_pages[9].find_all('span', 'd2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em iv3no6db jq4qci2q a3bd9o3v b1v8xokw m9osqain')

num_comment

[<span class="d2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em iv3no6db jq4qci2q a3bd9o3v b1v8xokw m9osqain" dir="auto">댓글 5개</span>,
 <span class="d2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em iv3no6db jq4qci2q a3bd9o3v b1v8xokw m9osqain" dir="auto">공유 3회</span>]

In [94]:
len(num_comment)

2

In [95]:
# 댓글 수 크롤링
num_comment = info_pages[4].find_all('span', 'd2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em iv3no6db jq4qci2q a3bd9o3v b1v8xokw m9osqain')

num_comment

[]

In [22]:
num_comment == None

True

In [87]:
# 공유 수 크롤링
num_share = info_pages[3].find('span', 'd2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em iv3no6db jq4qci2q a3bd9o3v b1v8xokw m9osqain')

num_share

<span class="d2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em iv3no6db jq4qci2q a3bd9o3v b1v8xokw m9osqain" dir="auto">공유 11회</span>

In [13]:
info_comment[0]

<span class="d2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em iv3no6db jq4qci2q a3bd9o3v b1v8xokw m9osqain" dir="auto">공유 2회</span>

In [59]:
info_comment[0].text[3:-1]

'2'

In [76]:
# 팔로워수, 글 날짜, 글 내용, 좋아요 수, 댓글 수, 공유 수 리스트화
num_follower_list = []
num_date_list = []
contents_list = []
num_like_list = []
num_comment_list = []
num_share_list = []

# 팔로워 수 수집
num_follower = soup.find('span', 'd2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em iv3no6db jq4qci2q a3bd9o3v b1v8xokw oo9gr5id hzawbc8m')
temp = float(num_follower.text[4:-2]) * 10000
num_follower_list.append(temp)
for i in range(len(info_pages)-1):
    num_follower_list.append(0)

# 날짜 수집
for date in info_dates_span:
    temp = date.find('span').text
    num_date_list.append(temp)

# 글 내용 수집
for content in content_pages:
    temp = content.text
    contents_list.append(temp)

# 좋아요 수 수집
for like in info_pages:
    temp = like.find('span', 'pcp91wgn')
    temp = int(temp.text)
    num_like_list.append(temp)

# 댓글 수 수집
for comment in info_pages:
    temp = comment.find('span', 'd2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em iv3no6db jq4qci2q a3bd9o3v b1v8xokw m9osqain')
    if temp == None:
        num_comment_list.append(0)
    else:
        temp = int(temp.text[3:-1])
        num_comment_list.append(temp)

# 공유 수 수집
for share in info_pages:
    temp = share.find('span', 'd2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em iv3no6db jq4qci2q a3bd9o3v b1v8xokw m9osqain')
    if temp == None:
        num_share_list.append(0)
    else:
        temp = int(temp.text[3:-1])
        num_share_list.append(temp)

print(len(num_follower_list), len(num_date_list), len(contents_list))
print(len(num_like_list), len(num_comment_list), len(num_share_list))

13 13 13
13 13 13


In [78]:
# 수집 데이터를 데이터 셋으로 만들기
df = {'날짜':num_date_list, '글_내용':contents_list, '좋아요_수':num_like_list,
      '댓글수':num_comment_list, '공유수':num_share_list, '팔로워_수':num_follower_list}
data = pd.DataFrame(df)
data

,날짜,글_내용,좋아요_수,댓글수,공유수,팔로워_수
0,3월 4일 오후 6:21,Gyeonggi Province’s Pocheon City provides fore...,17,0,0,99000.0
1,3월 4일 오전 10:58,Catch this week's #GTV English #News for the l...,8,0,0,0.0
2,3월 2일 오후 5:04,Gyeonggi Province promotes local tourism with ...,22,2,2,0.0
3,2월 28일 오후 5:49,Gyeonggi Province is taking the initiative in ...,52,11,11,0.0
4,2월 25일 오전 10:19,Catch this week's #GTV English #News for the l...,11,0,0,0.0
5,2월 24일 오후 3:00,Hwaseong Fortress of Suwon City,50,10,10,0.0
6,2월 22일 오후 5:51,"Gwacheon City of Gyeonggi Province, known as t...",17,0,0,0.0
7,2월 21일 오후 2:07,Catch this week's #GTV English #News for the l...,18,0,0,0.0
8,2월 18일 오후 3:00,Gyeonggi Province unveils Korea’s first virtua...,27,1,1,0.0
9,2월 15일 오전 1:00,Famous for family trips: Yangtte Farm in Gyeon...,40,5,5,0.0


In [79]:
# 크롤링 데이터 파일로 출력
file_name = str(datetime.today().strftime("%Y%m%d-%H%M%S"))
file_name = 'facebook_' + file_name + '.xlsx'
path = './crawling_data/facebook/'
data.to_excel(path+file_name, index=False, encoding='utf-8')

# 웹페이지 크롤링

In [80]:
from bs4 import BeautifulSoup as bs
import time
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [96]:
import pandas as pd

In [103]:
def fb_crawling(num_scrolldown):
    
    # 페이지 스크롤 다운
    for i in range(num_scrolldown):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)

    # 페이지 크롤링
    soup = bs(driver.page_source, 'lxml')
    print(soup.title.text)

    # 팔로워 수 수집
    num_follower = soup.find('span', 'd2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em iv3no6db jq4qci2q a3bd9o3v b1v8xokw oo9gr5id hzawbc8m')

    # 글 날짜 수집
    info_dates_span = soup.find_all('a', 'oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl gmql0nx0 gpro0wi8 b1v8xokw')

    # 글 내용 크롤링
    content_pages = soup.find_all('div', class_='kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x c1et5uql ii04i59q')

    # 좋아요, 댓글, 공유 수 정보 나온 페이지 수집
    info_pages = soup.find_all('div', 'bp9cbjyn m9osqain j83agx80 jq4qci2q bkfpd7mw a3bd9o3v kvgmc6g5 wkznzc2l oygrvhab dhix69tm jktsbyx5 rz4wbd8a osnr6wyh a8nywdso s1tcr66n')

    # 팔로워수, 글 날짜, 글 내용, 좋아요 수, 댓글 수, 공유 수 리스트화
    num_follower_list = []
    num_date_list = []
    contents_list = []
    num_like_list = []
    num_comment_list = []
    num_share_list = []

    # 팔로워 수 수집
    num_follower = soup.find('span', 'd2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em iv3no6db jq4qci2q a3bd9o3v b1v8xokw oo9gr5id hzawbc8m')
    temp = float(num_follower.text[4:-2]) * 10000
    num_follower_list.append(temp)
    for i in range(len(info_pages)-1):
        num_follower_list.append(0)

    # 날짜 수집
    for date in info_dates_span:
        temp = date.find('span').text
        num_date_list.append(temp)

    # 글 내용 수집
    for content in content_pages:
        temp = content.text
        contents_list.append(temp)

    # 좋아요 수 수집
    for like in info_pages:
        temp = like.find('span', 'pcp91wgn')
        temp = int(temp.text)
        num_like_list.append(temp)

    # 댓글 수, 공유 수 수집
    for comment in info_pages:
        temp = comment.find_all('span', 'd2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em iv3no6db jq4qci2q a3bd9o3v b1v8xokw m9osqain')
        if len(temp) == 1:
            for one in temp:
                if '댓글' in one.text:
                    comment_one = int(one.text[3:-1])
                    num_comment_list.append(comment_one)
                    num_share_list.append(0)
                elif '공유' in one.text:
                    comment_share = int(one.text[3:-1])
                    num_share_list.append(comment_share)
                    num_comment_list.append(0)
        elif len(temp) == 2:
            for one in temp:
                if '댓글' in one.text:
                    comment_one = int(one.text[3:-1])
                    num_comment_list.append(comment_one)
                elif '공유' in one.text:
                    comment_share = int(one.text[3:-1])
                    num_share_list.append(comment_share)
        else:
            num_share_list.append(0)
            num_comment_list.append(0)

    print('팔로워 수 리스트 원소수 : ',len(num_follower_list))
    print('팔로워 수 리스트 원소수 : ',len(num_date_list))
    print('글 내용 리스트 원소수 : ',len(contents_list))
    print('좋아요 수 리스트 원소수 : ',len(num_like_list))
    print('댓글 수 리스트 원소수 : ',len(num_comment_list))
    print('공유 수 리스트 원소수 : ',len(num_share_list))


    # 수집 데이터를 데이터 셋으로 만들기
    df = {'날짜':num_date_list, '글_내용':contents_list, '좋아요_수':num_like_list,
        '댓글수':num_comment_list, '공유수':num_share_list, '팔로워_수':num_follower_list}
    data = pd.DataFrame(df)

    # 수집한 데이터 파일료 출력
    file_name = str(datetime.today().strftime("%Y%m%d-%H%M%S"))
    file_name = 'facebook_' + file_name + '.xlsx'
    path = './crawling_data/facebook/'
    data.to_excel(path+file_name, index=False, encoding='utf-8')

    return data

In [104]:
# 크롤링할 웹사이트 주소
url = 'https://www.facebook.com/GyeonggiKorea.en'

# webdriver 로드
driver = webdriver.Chrome('chromedriver')
driver.get(url)
driver.implicitly_wait(5)

# 창 최대화
driver.maximize_window()
time.sleep(1)

In [105]:
# fb_crawling(page_down 누르는 횟수)
num_scrolldown = 3
data = fb_crawling(num_scrolldown)

Gyeonggi Korea | Facebook
팔로워 수 리스트 원소수 :  13
팔로워 수 리스트 원소수 :  13
글 내용 리스트 원소수 :  13
좋아요 수 리스트 원소수 :  13
댓글 수 리스트 원소수 :  13
공유 수 리스트 원소수 :  13


In [106]:
data

,날짜,글_내용,좋아요_수,댓글수,공유수,팔로워_수
0,3월 4일 오후 6:21,Gyeonggi Province’s Pocheon City provides fore...,17,0,0,99000.0
1,3월 4일 오전 10:58,Catch this week's #GTV English #News for the l...,8,0,0,0.0
2,3월 2일 오후 5:04,Gyeonggi Province promotes local tourism with ...,22,0,2,0.0
3,2월 28일 오후 5:49,Gyeonggi Province is taking the initiative in ...,52,0,11,0.0
4,2월 25일 오전 10:19,Catch this week's #GTV English #News for the l...,11,0,0,0.0
5,2월 24일 오후 3:00,Hwaseong Fortress of Suwon City,50,0,10,0.0
6,2월 22일 오후 5:51,"Gwacheon City of Gyeonggi Province, known as t...",17,0,0,0.0
7,2월 21일 오후 2:07,Catch this week's #GTV English #News for the l...,18,0,0,0.0
8,2월 18일 오후 3:00,Gyeonggi Province unveils Korea’s first virtua...,27,0,1,0.0
9,2월 15일 오전 1:00,Famous for family trips: Yangtte Farm in Gyeon...,40,5,3,0.0


# 코드 마지막